In [2]:
# loading and normalizing cifar10
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from torchsummary import summary

NameError: name 'nn' is not defined

In [3]:

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
                                        
                                        
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [12]:
## creating a function that initializes weights
def init_weight_conv(m):
    if type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight) #he initialize, can use xavier instead
        #m.bias.data.fill_(0.001) # optional bias
        
def init_weight_linear(m):
    if type(m) == nn.Linear:
        torch.nn.init.kaiming_normal_(m.weight)
        #m.bias.data.fill_(0.001)
    

In [44]:
# inception style module
class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001), # outchannels match previous conv layer
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding = 1),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_5x5 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU(),
            nn.Conv2d(8, 8, 5, padding = 2),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.max_pool_conv_1x1 = nn.Sequential(
            nn.MaxPool2d(3, stride = 1, padding=1), # window size 3x3, stride and padding = 1 to ensure dimensionality matches
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_1x1 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.fc1 = nn.Linear(8192, 10) 
        #self.dropout = nn.Dropout(0.5)
        #self.fc2 = nn.Linear(32, 10)
        
    def forward(self, x):
        x1 = self.conv_1x1(x)
        x2 = self.conv_3x3(x)
        x3 = self.conv_5x5(x)
        x4 = self.max_pool_conv_1x1(x)
        x = torch.cat([x1,x2,x3,x4], 1) # concatenate all layers
        x = F.avg_pool2d(x, stride = 2, kernel_size= 3)
        x = x.view(x.size(0), -1) # flatten
        x = self.fc1(x)
        #x = self.dropout(x)
        #x = self.fc2(x)
        return x
        
net = convnet()
                        

In [36]:
# fully convolutional network
class fullconv_net(nn.Module):
    def __init__(self):
        super(fullconv_net, self).__init__()
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001), # outchannels match previous conv layer
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding = 1),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_5x5 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU(),
            nn.Conv2d(8, 8, 5, padding = 2),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.max_pool_conv_1x1 = nn.Sequential(
            nn.MaxPool2d(3, stride = 1, padding=1), # window size 3x3, stride and padding = 1 to ensure dimensionality matches
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_1x1 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.fc1 = nn.Linear(8192, 10) 
        #self.dropout = nn.Dropout(0.5)
        #self.fc2 = nn.Linear(32, 10)
        self.conv = nn.Conv2d(4*8, 10, kernel_size=1)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
    def forward(self, x):
        x1 = self.conv_1x1(x)
        x2 = self.conv_3x3(x)
        x3 = self.conv_5x5(x)
        x4 = self.max_pool_conv_1x1(x)
        x = torch.cat([x1,x2,x3,x4], 1) # concatenate all layers
        x = F.relu(self.conv(x)) # conv filters to get to 10 channels
        x = self.avgpool(x) # adaptive avg pooling to get 10x1x1 output
        x = x.view(x.size(0), -1) # flatten
        return x
        
net = fullconv_net()
                        

In [29]:
# convolutional network
class convnet(nn.Module):
    def __init__(self):
        super(convnet, self).__init__()
        self.conv_3x3 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001), # outchannels match previous conv layer
            nn.ReLU(),
            nn.Conv2d(8, 8, 3, padding = 1),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_5x5 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU(),
            nn.Conv2d(8, 8, 5, padding = 2),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.max_pool_conv_1x1 = nn.Sequential(
            nn.MaxPool2d(3, stride = 1, padding=1), # window size 3x3, stride and padding = 1 to ensure dimensionality matches
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.conv_1x1 = nn.Sequential(
            nn.Conv2d(3, 8, 1, padding = 0),
            nn.BatchNorm2d(8, eps=0.001),
            nn.ReLU()
        )
        self.fc1 = nn.Linear(4*8, 32) 
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(32, 10)
        #self.conv = nn.Conv2d(4*8, 10, kernel_size=1)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        
    def forward(self, x):
        x1 = self.conv_1x1(x)
        x2 = self.conv_3x3(x)
        x3 = self.conv_5x5(x)
        x4 = self.max_pool_conv_1x1(x)
        x = torch.cat([x1,x2,x3,x4], 1) # concatenate all layers
        #x = F.relu(self.conv(x)) # conv filters to get to 10 channels
        x = self.avgpool(x) # adaptive avg pooling to get 10x1x1 output
        x = x.view(x.size(0), -1) # flatten to 
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
        
net = convnet()
                        

In [37]:
# visualize network
#for i, weights in enumerate(list(net.parameters())):
#    print('i:',i,'weights:',weights.size())

#print(net)


summary(net, (3, 32, 32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]              32
       BatchNorm2d-2            [-1, 8, 32, 32]              16
              ReLU-3            [-1, 8, 32, 32]               0
            Conv2d-4            [-1, 8, 32, 32]              32
       BatchNorm2d-5            [-1, 8, 32, 32]              16
              ReLU-6            [-1, 8, 32, 32]               0
            Conv2d-7            [-1, 8, 32, 32]             584
       BatchNorm2d-8            [-1, 8, 32, 32]              16
              ReLU-9            [-1, 8, 32, 32]               0
           Conv2d-10            [-1, 8, 32, 32]              32
      BatchNorm2d-11            [-1, 8, 32, 32]              16
             ReLU-12            [-1, 8, 32, 32]               0
           Conv2d-13            [-1, 8, 32, 32]           1,608
      BatchNorm2d-14            [-1, 8,

In [38]:
# apply initializers
net.apply(init_weight_conv)
net.apply(init_weight_linear)

fullconv_net(
  (conv_3x3): Sequential(
    (0): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (conv_5x5): Sequential(
    (0): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
    (1): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(8, 8, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (max_pool_conv_1x1): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (1): Conv2d(3, 8, kernel_size=(1, 1), stride=(1, 1))
    (2): BatchNorm2d(8, eps=0.001, momentum=0.1, affine=True, track_runnin

In [39]:
# define loss and optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate) #net.parameters
#optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, nesterov=True)

In [43]:
# modified model training to keep track of train/val loss
n_epochs = 2

for epoch in range(n_epochs):
    running_loss = 0.0
    total_train_loss = 0.0
    for i, train_data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = train_data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print loss per n minibatches
        running_loss += loss.item()
        total_train_loss += loss.item()
        if i % 50 == 49:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
    
    # keep track of loss in test dataset 
    correct = 0
    total = 0
    total_test_loss = 0.0
    with torch.no_grad():
        for test_data in testloader:
            test_images, test_labels = test_data
            test_outputs = net(test_images)
            test_loss = criterion(test_outputs, test_labels)
            total_test_loss += test_loss.item()
            _, predicted = torch.max(test_outputs.data, 1)
            total += test_labels.size(0)
            correct += (predicted == test_labels).sum().item()


    
    
    # for printing average loss every epoch
    print("===> Epoch {} Complete: Train Avg. Loss: {:.4f}".format(epoch+1, total_train_loss / len(trainloader)))
    print("===> Epoch {} Complete: Test Avg. Loss: {:.4f}".format(epoch+1, total_test_loss / len(testloader)))
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
print('Finished Training')

[1,    50] loss: 1.848
[1,   100] loss: 1.872
[1,   150] loss: 1.876
[1,   200] loss: 1.889
[1,   250] loss: 1.865
[1,   300] loss: 1.914
[1,   350] loss: 1.861
[1,   400] loss: 1.863
[1,   450] loss: 1.844
[1,   500] loss: 1.853
[1,   550] loss: 1.847
[1,   600] loss: 1.858
[1,   650] loss: 1.830
[1,   700] loss: 1.840
[1,   750] loss: 1.827
[1,   800] loss: 1.849
[1,   850] loss: 1.842
[1,   900] loss: 1.845
[1,   950] loss: 1.828
[1,  1000] loss: 1.847
[1,  1050] loss: 1.812
[1,  1100] loss: 1.806
[1,  1150] loss: 1.804
[1,  1200] loss: 1.799
[1,  1250] loss: 1.833
[1,  1300] loss: 1.808
[1,  1350] loss: 1.814
[1,  1400] loss: 1.817
[1,  1450] loss: 1.804
[1,  1500] loss: 1.844
[1,  1550] loss: 1.823
===> Epoch 1 Complete: Train Avg. Loss: 1.8402
===> Epoch 1 Complete: Test Avg. Loss: 1.8017
Accuracy of the network on the 10000 test images: 34 %
[2,    50] loss: 1.795
[2,   100] loss: 1.781
[2,   150] loss: 1.796
[2,   200] loss: 1.819
[2,   250] loss: 1.788
[2,   300] loss: 1.781
[